# ❄️ ICE3: Spatial Math for Python

## A note on this document
This document is known as a Jupyter notebook; it is used in academia and industry to allow text and executable code to coexist in a very easy to read format. Blocks can contain text or code, and for blocks containing code, press `Shift + Enter` to run the code. Earlier blocks of code need to be run for the later blocks of code to work.

## Introduction
This lab assignment will introduce the spatialmath package that will be used throughout the semester to control robotic arms.

### Documentation
This lab has been adapted from the Spatial Math for Python Documentation https://petercorke.github.io/spatialmath-python/.  Refer to the follwing links for more details.

https://github.com/petercorke/spatialmath-python
https://petercorke.github.io/spatialmath-python/


## Procedure

### Transformation Matrices in 2D

First, import the **low-level** transform functions

In [ ]:
from spatialmath.base import *  # import low-level functions
import numpy as np

In [ ]:
# We can create a rotation matrix in 2-dimensional space
R = rot2(np.pi/4) # the angle is specified in radians.
print(R)

# The object type of R can be found by
print(type(R))
# R is a numpy.ndarray object

# We can find the determinant of the matrix such as
d = np.linalg.det(R)
print(d)

As it is expected, R is a 2 by 2 matrix that is a `numpy.ndarray` object.  We can also create a homogeneous transformation matrix in 2-D that is a 3 by 3 matrix.

In [ ]:
# Homogeneous transformation matrix for a rotation of 30 deg
T = trot2(30, 'deg')
print(T)

# Create a homogeneous transformation in 2-D that represents 
# a translation of (1,2) followed by a rotation of 45 deg
T1 = transl2(1,2)@trot2(np.pi/4)   # It is trans el-two not trans one-two
print(T1)

type(T1)

**Note**: `*` is for element-wise multiplication whereas `@` is for matrix multiplication.

The function `transl2` creates a relative pose with a finite translation but zero rotation, while `trot2` creates a relative pose with a finite rotation but zero translation.  We can plot this, relative to the world coordinate frame, by

In [ ]:
trplot2(T1, frame='A')

If it does not display a plot in PyCharm, you need to install matplotlib package.  Go to File > Settings > Project > Python Interpreter. Click + to install matplotlib. Once installation is complete add to your code
```
import matplotlib.pyplot as plt
```
and after the `trplot2()` function add
```
plt.show()
```

Use `plotvol2` to modify the plot area

In [ ]:
trplot2(T1, frame='A')
plotvol2([0,2,0,3], grid=True)  # axis dimension: x = [0,2], y=[0,3] 

We can run `help` function to learn details of these functions by

In [ ]:
help(trplot2)

We can create another relative pose which is a displacement of (2,1) and zero rotation

In [ ]:
T2 = transl2(2,1)
print(T2)

Now we can compose the two relative poses

In [ ]:
T3 = T1@T2
print(T3)

and plot them as

In [ ]:
trplot2(T1, frame='A', width=1)
trplot2(T2, frame='B', color='red', width=1)
trplot2(T3, frame='C', color='green', width=1)
plotvol2([0,4,0,5], grid=True)

**Notice that the displacement of (2,1) has been applied with respect to frame {A}.** It is important to note that the final displacement is not (3,3) because the displacement is with respect to the rotated coordinate frame. The noncommutativity of composition is clearly demonstrated by 

In [ ]:
T4 = T2@T1
trplot2(T1, frame='A', width=1)
trplot2(T2, frame='B', color='red', width=1)
trplot2(T3, frame='C', color='green', width=1)
trplot2(T4, frame='D', color='black')
plotvol2([0,4,0,5], grid=True)

and we can see that Frame {D} is different to Frame {C} as shown in Figure 1. 

Now we can define a point $[3,2]^\top$ relative to the world frame (Frame {0})

In [ ]:
P_0 = np.array([3,2]).reshape([2,1])  # it must be a 2 by 1 column vector 
print(P_0)

which is a column vector.

We can determine the coordinate of the point with respect to {A} by
$$^0\tilde{\mathbf{p}} = {^0T}_A {^A\tilde{\mathbf{p}}} $$
and then rearrange as
$${^A\tilde{\mathbf{p}}} = {^AT_0} {^0\tilde{\mathbf{p}}} = {^0T^{-1}_A} {^0\tilde{\mathbf{p}}} $$
Substituting numeric values

In [ ]:
p_0 = e2h(P_0)  # convert from Euclidean to homogeneous form
print(p_0)
p_A = np.linalg.inv(T1)@p_0
print(p_A)

where we first convert the Euclidean point coordinates to homogeneous form by appending a one. The result is also in homogeneous form and has a negative y-coordinate in frame {A}.  Using `h2e` we could also have expressed this as 

In [ ]:
P_A = h2e(np.linalg.inv(T1)@p_0)
print(P_A)

You can use `set_printoptions` to change the way foating point numbers and numpy arrays are displayed

In [ ]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
print(P_A)

###	Transformation Matrices in 3D

The spatial maths package provides functions to compute rotation matrices in 3D, for example $R_X(\theta)$ is

In [ ]:
R = rotx(np.pi/2)
print(R)

The function `rotx` returns the rotation matrix that rotates a point around the x-axis for an input in radians. The functions `roty` and `rotz` compute $R_Y(\theta)$ and $R_Z(\theta)$, respectively.  The corresponding coordinate frame can be displayed graphically

In [ ]:
trplot(R)

To illustrate compounding of rotations we will rotate the frame again, this time around its y-axis

In [ ]:
R = rotx(np.pi/2)@roty(90, 'deg')
print(R)
trplot(R)

The non-commutativity of rotation can be shown by reversing the order of the rotations:

In [ ]:
R = roty(np.pi/2)@rotx(np.pi/2)
print(R)
trplot(R)

The ZYX Euler angles $R=R_Z(\psi)R_Y(\theta)R_X(\phi)$ is commonly used in robotic arms. The Euler angles are the 3-vector $(\psi, \theta, \phi)$. For example, to compute the equivalent rotation matrix for $(60^\circ, 45^\circ, 30^\circ)$ we write


In [ ]:
Rzyx = rotz(np.pi/6)@roty(np.pi/4)@rotx(np.pi/3)
print(Rzyx)

or more conveniently

In [ ]:
Rzyx = rpy2r(np.pi/3, np.pi/4, np.pi/6)  # roll-pitch-yaw to rotation matrix
print(Rzyx)

The function `rpy2r(alpha, beta, gamma)` transforms roll-pitch-yaw angles to a rotation matrix. The default is ZYX Euler angles - rotate by gamma about the z-axis, then by beta about the new y-axis, then by alpha about the new x-axis.  The inverse is

In [ ]:
rpy = tr2rpy(Rzyx)
print(rpy)

in radians or

In [ ]:
rpy = tr2rpy(Rzyx, unit='deg')
print(rpy)

in degrees.  To compute the rotation matrix for XYZ Euler angles with $(60^\circ, 45^\circ, 30^\circ)$ we write

In [ ]:
Rxyz = rotx(np.pi/6)@roty(np.pi/4)@rotz(np.pi/3)
print(Rxyz)

or

In [ ]:
Rxyz = rpy2r(np.pi/3, np.pi/4, np.pi/6, order='xyz')
print(Rxyz)

The inverse is

In [ ]:
rpy = tr2rpy(R, unit='deg', order='xyz')
print(rpy)

A homogeneous transformation matrix can be implemented by

In [ ]:
T = rpy2tr(0, 90, 0, unit='deg', order='zyx')
print(T)

which is a 4 by 4 matrix. We can extract the Euler angles by

In [ ]:
rpy = tr2rpy(T, unit='deg')
print(rpy)

The `trplot` function supports various plotting styles as follows.

In [ ]:
trplot( transl(1,2,3), frame='A', rviz=True, width=1)
trplot( transl(3,1, 2), color='red', width=3, frame='B')
trplot( transl(4, 3, 1)@trotx(np.pi/3), color='green', frame='c', dims=[0,4,0,4,0,4])

We can visualize a rotation more powerfully using the `tranimate` function which animates a rotation

In [ ]:
# Jupyter notebook cannot properly animate transformation matrices. 
# You must run it in PyCharm. 
T = transl(4, 3, 4)@trotx(2)@troty(-2)
tranimate(T, frame='A', dims=[0, 5], nframes=200)

showing the world frame rotating into the specified coordinate frame.  You can also save the animation to a file by

In [ ]:
tranimate(T, frame='A', dims=[0, 5], nframes=200, movie='out.mp4')

### High-level classes
The spatialmath package supports high-level classes that abstract the low-level numpy arrays into objects.  These objects obey the rules associated with the mathematical groups SO(2), SE(2), SO(3), SE(3), twists, and quaternioins.

To create an object representing a rotation of 90 degrees about the x-axis is

In [1]:
import numpy as np
from spatialmath import *  # import high-lelel spatialmath functions

R1 = SO3.Rx(np.pi/2)
print(R1)

   1         0         0         
   0         0        -1         
   0         1         0         



The object type of R1 can be found by

In [2]:
type(R1)

spatialmath.pose3d.SO3

The functions we used earlier, such as `rotx`, `roty`, `rotz`, `rpy2r`, returned the numpy `array` type. The compounding of rotations can be found by

In [3]:
Rx = SO3.Rx(np.pi/2)
Ry = SO3.Ry(np.pi/2)
R = Rx * Ry    # We use * not @ for matrix multiplication for SO3 objects.
print(R)

   0         0         1         
   1         0         0         
   0         1         0         



Note that we use `*` for matrix multiplication. We can find the corresponding Euler angles

In [8]:
float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})
print(R.eul('deg'))  # eul Returns ZYZ Euler angels
print(R.rpy('deg'))  # Returnz

[-0.00 90.00 90.00]
[90.00 0.00 90.00]


A pure translation in 3-D can be represented by

In [ ]:
T = SE3(1,2,3)  # Special Euclidean group.
print(T)

which is a 4 by 4 matrix. A transformation matrix representing a translation followed by a rotation can be obtained by

In [ ]:
T = SE3(1,2,3)*SE3.Rx(30, 'deg')
print(T)

A rotation followed by a translation in 3-D can be obtained by

In [ ]:
T = SE3.Rx(30, 'deg')*SE3(1,2,3)
print(T)

which is different than a translation followed by a rotation.  **Note** that we used `SE3.Rx()` not `SO3.Rx()`.  `SE3.Rx()` returns a 4 by 4 SE(3) matrix whereas `SO3.Rx()` returns 3 by 3 SO(3) matrix.  We can print the pose (position and orientation) by

In [ ]:
T.printline()

We can plot the transformation by

In [ ]:
T.plot()

***You can find class/function references of the spatial maths package at
https://petercorke.github.io/spatialmath-python/ ***

## Deliverables
Create a folder named Lab1 in which you add exercise1.py, exercise2.py, and so on for the following exercises. 

**Submit your code to Bitbucket and provide the outputs in Gradescope.**

### Exercise 1
Using the ZYX Euler angles, find the rotation matrix $R_{ZYX}(\psi,\theta,\phi)$ for $\psi=45^\circ, \theta=60^\circ, \phi=45^\circ$.

(i) Use low-level spatial math functions to find $R_{ZYX}$ and the Euler angles from $R_{ZYX}$.

(ii) Use high-level spatial math functions to find $R_{ZYX}$ and the Euler angles from $R_{ZYX}$.

Did you get the same Euler angels, $\psi$, $\theta$, and $\phi$?  If not, explain why.  

### Exercise 2
Using the ZYX Euler angles, find the rotation matrix $R_{ZYX}(\psi,\theta,\phi)$ for $\psi=45^\circ, \theta=90^\circ, \phi=45^\circ$. 

(i) Use low-level spatial math functions to find $R_{ZYX}$ and the Euler angles from $R_{ZYX}$.

(ii) Use high-level spatial math functions to find $R_{ZYX}$ and the Euler angles from $R_{ZYX}$.

Did you get the same Euler angels, $\psi$, $\theta$, and $\phi$?  If not, explain why.  

### Exercise 3
Find $\phi$, $\theta$, and $\psi$ for YZY Euler angles, $R=R_Y(\phi)R_Z(\theta)R_Y(\psi)$, that generate the blue frame shown in figure below.  The blue frame has been rotated with respect to the blakc frame.  Use SE3.Rx, SE3.Ry and SE3.Rz.
<div>
<br>
    <img src="figs/Ex3.png" width="450"/>
</div>
Use the following code snippet.

In [ ]:
from spatialmath.base import *
from spatialmath import *
import matplotlib.pyplot as plt

# add your code here.





print(T)
T.plot()
plotvol3([-2, 2, -2, 2, -2, 2], grid=True)
plt.show()
